### Configure Ollama with structured outputs ###

In [2]:
import os
import pandas as pd
import numpy as np
import logging
import time
from pathlib import Path
from pydantic import BaseModel
import json
import dotenv
from IPython.display import clear_output
from tqdm import tqdm

logger = logging.getLogger(__name__)

import ollama
from ollama import Client

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.ollamamodel import Ollama, OllamaModel
from llmt.openaimodel import MentalHealth, OutpatientServices, InpatientServices, create_messages
from llmt.performance import Performance
from llmt.filetools import FileOP
# print(llmt.__version__)

### Custom Ollama Client ###

In [3]:
# Parameters
client = Client(
  host='http://ollama:11434',
  headers={'x-some-header': 'some-value'}
)
model = 'llama2:13b'
#client.pull(model)
ollama_models = client.list().get('models')
ollama_model_list = [model['model'] for model in ollama_models]
print(ollama_model_list)

# Use class method
ollama_model_list = Ollama().models
print(ollama_model_list)

['llama2:13b', 'llama2:7b']
['llama2:13b', 'llama2:7b']


In [3]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'

# Load the data from URL
url = f'https://dsets.s3.us-east-1.amazonaws.com/{test_file_name}'
test_file = FileOP().download_from_url(url=url, download_dir=data_dir)
df = pd.read_parquet(test_file)

# Filter the labeled data
df = df.loc[df['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df.head())
print(df.shape)

Created .parquet file.


,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2,0,0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0,0,0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0,0,0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1,0,1,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2,0,0,train


(187, 7)


### Structured Outputs ###

In [5]:
# Pick an example of a company that provides mental health services
mental_health = 1
inpatient = 1
id_list = list(df.loc[(df['mental_health'] == mental_health) & (df['inpatient'] == inpatient), 'id'].unique())
print(f'Found {len(id_list)} company ids')

# Pick a company ID
idx = 10
company_id = id_list[idx]
# company_id = '135432-64'
ser_id = df.loc[df['id'] == company_id].iloc[0]
display(ser_id)
name = ser_id['name']
description = ser_id['description']
print()
print(name)
print(description)

Found 62 company ids


id                                                       444148-30
name                                        Purpose Healing Center
description      Provider of alcohol and drug treatment centers...
mental_health                                                    1
inpatient                                                        1
outpatient                                                       1
dset                                                         train
Name: 116, dtype: object


Purpose Healing Center
Provider of alcohol and drug treatment centers intended for inpatient and outpatient programs. The company offers a variety of programs including medical detox, medication assisted treatment, and inpatient and outpatient rehab, patients, including dual diagnosis, mental health services, enabling patients to get a safe recovery environment away from drugs.


In [6]:
# Create an ollama class
api_endpoint = 'http://ollama:11434'
def create_client():
    client = None
    try:
        client = ollama.Client(host=api_endpoint,
                               headers={'x-some-header': 'some-value'})
    except Exception as e:
        logger.error(f'Error: {e}')
    return client

client = create_client()

In [7]:
version = 3
variable = 'mental_health'
pred_col = 'pred_mh'
mh_prompt_name = f'{variable}_system_{str(version).zfill(2)}'
system_prompt = Prompt().load(prompt_name=mh_prompt_name)

In [8]:
print(system_prompt)

You are an AI system assisting a healthcare policy researcher in identifying whether a business qualifies as a medical facility or organization that provides direct mental or behavioral healthcare services to human patients.
A qualifying business must meet all of the following criteria:

It is a specialized healthcare facility or organization, such as a general hospital, mental health clinic, psychiatric hospital, counseling center, or behavioral health treatment center.  
It provides direct services to human patients, in-person or online, including assessments, diagnoses, therapy (individual, group, or family), psychiatric evaluations, medication management, and/or crisis intervention.  
Services are delivered by licensed mental health professionals such as psychiatrists, psychologists, counselors, clinical social workers, or psychiatric nurse practitioners.

Exclude any business that falls into the following categories, even if they contribute to mental health solutions:

Pharmaceuti

In [9]:
user_prompt = process_prompt(f"""
                        The organization {name} is described as: {description} 
                        Does this organization provide {variable} healthcare services?
                        """)
print(user_prompt)

The organization Purpose Healing Center is described as: Provider of alcohol and drug treatment centers intended for inpatient and outpatient programs. The company offers a variety of programs including medical detox, medication assisted treatment, and inpatient and outpatient rehab, patients, including dual diagnosis, mental health services, enabling patients to get a safe recovery environment away from drugs. 
Does this organization provide mental health healthcare services?


In [10]:
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
print(messages)

[{'role': 'system', 'content': 'You are an AI system assisting a healthcare policy researcher in identifying whether a business qualifies as a medical facility or organization that provides direct mental or behavioral healthcare services to human patients.\nA qualifying business must meet all of the following criteria:\n\nIt is a specialized healthcare facility or organization, such as a general hospital, mental health clinic, psychiatric hospital, counseling center, or behavioral health treatment center.  \nIt provides direct services to human patients, in-person or online, including assessments, diagnoses, therapy (individual, group, or family), psychiatric evaluations, medication management, and/or crisis intervention.  \nServices are delivered by licensed mental health professionals such as psychiatrists, psychologists, counselors, clinical social workers, or psychiatric nurse practitioners.\n\nExclude any business that falls into the following categories, even if they contribute t

In [ ]:
# Create messages using the Prompt class


In [11]:
# client = create_client()
om = Ollama()
model = 'llama2:7b'
temperature = 0
client = om.create_client()

response = client.chat(messages=messages,
                       model=model,
                       format=MentalHealth.model_json_schema(),
                       options={'temperature': temperature})

def response2message(response):
    output = None
    if response.get('done', None):
        if response.get('done_reason', None) == 'stop':
            output = response.message.content
            # clean up the response message
            output = output.\
                replace('\n', '').\
                replace('\t', '').\
                replace(' ','')
    return output

message = response2message(response)

In [12]:
print(message)

{"pred_mh":false,"pred_mh_score":0}


In [24]:
output = MentalHealth.model_validate_json(message)
output = output.model_dump()

response_dump = response.model_dump()

output.update({'done': response_dump.get('done'),
               'done_reason': response_dump.get('done_reason')})

print(message)
print(output)

{"pred_mh":false,"pred_mh_score":0}
{'pred_mh': False, 'pred_mh_score': 0.0, 'done': True, 'done_reason': 'stop'}


In [34]:
# Use the class methods
model = 'mistral-small3.1:24b'
temperature = 0
mh = OllamaModel(model=model).predict_mh(name=name, description=description, version=3, temperature=temperature)
ip = OllamaModel(model=model).predict_ip(name=name, description=description, version=1, temperature=temperature)
op = OllamaModel(model=model).predict_op(name=name, description=description, version=1, temperature=temperature)

print(user_prompt)
print()
print(mh)
print(ip)
print(op)

Error sending messages: model "mistral-small3.1:24b" not found, try pulling it first (status code: 404)


AttributeError: 'NoneType' object has no attribute 'update'

In [32]:
OllamaModel().models

['llama2:13b', 'llama2:7b']